In [64]:
from sqlalchemy import text, create_engine
import pandas as pd
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os
import numpy as np 
import re
import time
import fitz
import re

In [32]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [33]:
consultant_name_df = pd.read_excel('G:/Post Construction/metadata csvs/consultants.xlsx', encoding = 'utf-8-sig')
consultant_name_lst = consultant_name_df["consultant_name"].tolist()

In [34]:
with engine.connect() as conn: ## with context manager with, we don't have to close the connection everytime we run queries
    query1 = "SELECT * FROM pdfs;"
    df1 = pd.read_sql(query1, conn)

In [35]:
start = time.time()
main_consultant_name = []
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\t1nipun\AppData\Local\Tesseract-OCR\tesseract.exe'
for file in df1['pdfName']:
    pages = convert_from_path("G:/Post Construction/PCMR_All/" + file + '.pdf', 500, last_page= 1)
    for page in pages:
        filename = file +'_Page1.jpg'
        page.save("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + filename, 'JPEG')
        text = str(((pytesseract.image_to_string(Image.open("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + filename))))).lower().replace('\n\n',' ')
        consultant_name = []
        for name in consultant_name_lst:
            if name.lower() in text:
                consultant_name.append(name)
        main_consultant_name.append(consultant_name)
end = time.time()
print(f'Runtime is {end - start}')

Runtime is 1890.5476598739624


In [11]:
'''start = time.time()
main_consultant_name = []
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\t1nipun\AppData\Local\Tesseract-OCR\tesseract.exe'
for file in df1['pdfName']:
    pages = convert_from_path("G:/Post Construction/PCMR_All/" + file + '.pdf', 500)
    text = ""
    for i in range(0,2):
        pages[i].save("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + file +'_{}.jpg'.format(i), 'JPEG')
    if len(pytesseract.image_to_string(Image.open("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + file +'_{}.jpg'.format(1)))) == 0:
        text += pytesseract.image_to_string(Image.open("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + file +'_{}.jpg'.format(0))).lower()
    else:
        text += pytesseract.image_to_string(Image.open("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + file +'_{}.jpg'.format(i))).lower()
    consultant_name = []
    for name in consultant_name_lst:
        if name.lower() in text:
            consultant_name.append(name)
    main_consultant_name.append(consultant_name)
    print(file)
end = time.time()
print(f'Runtime is {end - start}')'''

In [104]:
df1['consultant_name'] = main_consultant_name
df2 = df1.copy()
mapping_dict = df2.groupby('filingId')['consultant_name'].apply(lambda x: x.values.tolist()).to_dict()

In [105]:
def flatten(lst_of_lsts):
    output = []
    for element in lst_of_lsts:
        if type(element) == list:
            output.extend(element)
    return list(set(output))

In [106]:
newdict = {key: flatten(value) for key, value in mapping_dict.items()}
newdict[775971] = ['Golder Associates']
newdict[786189] = ['TERA Environmental Consultants']
df2['consultant_name'] = df2['filingId'].map(newdict)

In [107]:
pdf_name = []
for line, row in enumerate(df2.itertuples(),1):  ## use head method with dataframe in order to slice the data when using itertuples() example df1.head(2).itertuples()
    if len(row.consultant_name) == 0:
        pdf_name.append(row.pdfName)
        try:
            pdf_file = open('G:/Dev/PCMR/pdf_files/' + row.pdfName + '.pdf', 'rb').read()
            pdf_doc = fitz.open('pdf', pdf_file)
        except:
            print('File: {} failed to open'.format(row.pdfName))
        if 'table of contents' not in row.xmlContent.lower() or len(pdf_doc.loadPage(1).getText("text")) == 0:
            page_content = pdf_doc.loadPage(2).getText("text").lower()
        else:
            page_content = pdf_doc.loadPage(1).getText("text").lower()
        consultant_name_step2 = []
        for name in consultant_name_lst:
            if name.lower() in page_content:
                consultant_name_step2.append(name)
        df2.set_value(row.Index, 'consultant_name', consultant_name_step2) # reference: https://stackoverflow.com/questions/43222878/iterate-over-pandas-dataframe-and-update-the-value-attributeerror-cant-set-a

mupdf: No default Layer config
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object


In [109]:
columns  = ['pdfId', 'pdfName', 'filingId', 'totalPages', 'application_id', 'submitter', 'company', 'consultant_name']
df2.to_csv('check_df3.csv', encoding = 'utf-8-sig', columns = columns)

In [ ]:
'''pdf_name = []
main_consultant_name_step2 = []
for line, row in enumerate(df2.itertuples(),1):  ## use head method with dataframe in order to slice the data when using itertuples() example df1.head(2).itertuples()
    if len(row.consultant_name) == 0:
        pdf_file = open('G:/Dev/PCMR/pdf_files/' + row.pdfName + '.pdf', 'rb')
        read_pdf = PyPDF2.PdfFileReader(pdf_file)
        if 'table of contents' not in row.xmlContent.lower():
            page_content = read_pdf.getPage(2).extractText().lower()
            Research = re.search(string,page_content)
            print(row.pdfName,Research)     
        pd
        text = row.xmlContent.lower().strip().split("table of contents")
        text_tuple = '_'.join(text[:1]), '_'.join(text[1:])
        text_before_TOC = text_tuple[0]
        consultant_name_step2 = []
        for name in consultant_name_lst:
            if name.lower() in text_before_TOC:
                consultant_name_step2.append(name)
        main_consultant_name_step2.append(consultant_name_step2)
                #df1.set_value(row.Index, 'consultant_name', name) # reference: https://stackoverflow.com/questions/43222878/iterate-over-pandas-dataframe-and-update-the-value-attributeerror-cant-set-a'''